In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import holidays
import pytz

from datetime import datetime
from scipy import stats

from catboost import *
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import ward, fcluster
from scipy.spatial.distance import pdist
from bayes_opt import BayesianOptimization

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from sklearn.model_selection import train_test_split
train = pd.read_csv('data/train_v2.csv', parse_dates=['date'])
test = pd.read_csv('data/test_non_anomaly_v2.csv', parse_dates=['date'])
y_train = train[['online_hours']]
X_train = train.drop(['driver_id', 'day', 'date', 'online_hours', 
                      'holiday', 'next_holiday', 'prev_holiday'],axis=1)
X_test = test.drop(['driver_id', 'day', 'date', 'online_hours', 
                      'holiday', 'next_holiday', 'prev_holiday'],axis=1)
y_test = test[['online_hours']]

In [4]:
X_train_subset, X_test_subset, y_train_subset, y_test_subset = train_test_split(X_train, y_train, 
                                                                                test_size=0.2, random_state=0)

# XGBoost 

In [5]:
class xgboost_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train.values
        self.y_train = y_train.values
        self.x_test = x_test.values
        self.y_test = y_test.values
        
    def clean_param(self, param) :
        booster_dict = {1:'gbtree', 2:'gblinear', 3:'dart'}
        params = {'random_state':0, 
                  'subsample':1, 
                  'colsample_bytree':1, 
                  'colsample_bylevel':1}
        
        params['objective'] = 'reg:squarederror'
        params['learning_rate'] = 0.1
        params['n_estimators'] = 100
        params['booster'] = booster_dict[int(param['booster'])]
        params['gamma'] = param['gamma']
        params['reg_alpha'] = param['reg_alpha']
        params['reg_lambda'] = param['reg_lambda']
        params['scale_pos_weight'] = param['scale_pos_weight']
        params['base_score'] = param['base_score']
        params['rate_drop'] = param['rate_drop']
        return params
        
    def evaluate(self,  
                 booster, 
                 gamma, 
                 reg_alpha, 
                 reg_lambda, 
                 scale_pos_weight, 
                 base_score, 
                 rate_drop):

        params = {}
        params['learning_rate'] = 0.1
        params['n_estimators'] = 100
        params['booster'] = booster
        params['gamma'] = gamma
        params['reg_alpha'] = reg_alpha
        params['reg_lambda'] = reg_lambda
        params['scale_pos_weight'] = scale_pos_weight
        params['base_score'] = base_score
        params['rate_drop'] = rate_drop
        params = self.clean_param(params)

        xgb_model = XGBRegressor(**params)
        xgb_model.fit(self.x_train, self.y_train)
        y_pred = xgb_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [6]:
xt = xgboost_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
xgbBO = BayesianOptimization(xt.evaluate, {'booster' : (1, 3),
                                            'gamma' : (0, 50),
                                            'reg_alpha' : (0, 50),
                                            'reg_lambda' : (0, 50),
                                            'scale_pos_weight' : (0, 50),
                                            'base_score' : (0, 50),
                                            'rate_drop': (0, 1)},
                            random_state = 1)

xgbBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | base_s... |  booster  |   gamma   | rate_drop | reg_alpha | reg_la... | scale_... |
-------------------------------------------------------------------------------------------------------------
|  1        | -3.543    |  20.85    |  2.441    |  0.005719 |  0.3023   |  7.338    |  4.617    |  9.313    |
|  2        | -1.926    |  17.28    |  1.794    |  26.94    |  0.4192   |  34.26    |  10.22    |  43.91    |
|  3        | -3.156    |  1.369    |  2.341    |  20.87    |  0.5587   |  7.019    |  9.905    |  40.04    |
|  4        | -1.932    |  48.41    |  1.627    |  34.62    |  0.8764   |  44.73    |  4.252    |  1.953    |
|  5        | -3.427    |  8.492    |  2.756    |  4.917    |  0.4211   |  47.89    |  26.66    |  34.59    |
|  6        | -3.423    |  15.78    |  2.373    |  41.73    |  0.01829  |  37.51    |  49.44    |  37.41    |
|  7        | -3.424    |  14.02    |  2.579    |  5.161    |  0.4479   |  45.43    |  14.68    |  14.39    |
|  8      

In [7]:
xgbBO.max

{'target': -1.914544397899434,
 'params': {'base_score': 6.501428605913883,
  'booster': 1.0387339157405941,
  'gamma': 33.94177664699455,
  'rate_drop': 0.21162811600005904,
  'reg_alpha': 13.277332968611312,
  'reg_lambda': 24.578657964016916,
  'scale_pos_weight': 2.668127255854019}}

# LGBM 

In [8]:
class lgbm_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {'boosting_type':'gbdt', 'class_weight':None, 'colsample_bytree':1.0, 
                  'importance_type':'split', 'learning_rate':0.1,
                  'min_child_samples':20, 'min_split_gain':0.0, 'n_estimators':100, 'objective':None,
                  'random_state':0, 'reg_alpha':0.0, 'reg_lambda':0.0, 'silent':True,
                  'subsample':1.0, 'subsample_for_bin':200000, 'subsample_freq':0}
        params['num_leaves'] = int(param['num_leaves'])
        params['min_child_weight'] = int(param['min_child_weight'])
        params['max_depth'] = int(param['max_depth'])
        params['learning_rate'] = 0.1
        params['min_data_in_bin'] = 1
        params['min_data'] = 1
        
        params['min_child_samples'] = int(param['min_child_samples'])
        params['bagging_fraction'] = param['bagging_fraction']
        params['lambda_l1'] = param['lambda_l1']
        params['lambda_l2'] = param['lambda_l2']

        return params
        
    def evaluate(self, min_child_weight, max_depth, num_leaves,
                min_child_samples, bagging_fraction, lambda_l1, lambda_l2):
        params = {'num_leaves':num_leaves, 
                  'min_child_weight':min_child_weight, 
                  'max_depth':max_depth,
                 'min_child_samples':min_child_samples,
                 'bagging_fraction' : bagging_fraction,
                 'lambda_l1' : lambda_l1,
                 'lambda_l2' : lambda_l2}
        
        params = self.clean_param(params)

        lgbm_model = LGBMRegressor(**params)
        lgbm_model.fit(self.x_train, self.y_train)
        y_pred = lgbm_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [9]:
lt = lgbm_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
lgbmBO = BayesianOptimization(lt.evaluate, {'min_child_weight': (0.01, 1),
                                              'max_depth': (1, 7),
                                              'num_leaves': (5, 50),
                                            'min_child_samples' :(10,50),
                                            'bagging_fraction' : (0.5,1),
                                            'lambda_l1' : (0,1),
                                            'lambda_l2' : (0,1)
                                           }, 
                             random_state=3)

lgbmBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | baggin... | lambda_l1 | lambda_l2 | max_depth | min_ch... | min_ch... | num_le... |
-------------------------------------------------------------------------------------------------------------
|  1        | -1.898    |  0.7754   |  0.7081   |  0.2909   |  4.065    |  45.72    |  0.8973   |  10.65    |
|  2        | -2.056    |  0.6036   |  0.05147  |  0.4408   |  1.179    |  28.27    |  0.6527   |  17.53    |
|  3        | -1.893    |  0.8381   |  0.5909   |  0.02398  |  4.353    |  20.37    |  0.421    |  17.76    |
|  4        | -1.895    |  0.8466   |  0.4405   |  0.1569   |  4.268    |  41.21    |  0.3133   |  14.99    |
|  5        | -1.894    |  0.694    |  0.9364   |  0.976    |  5.034    |  46.11    |  0.8473   |  22.01    |
|  6        | -1.905    |  0.5461   |  0.6534   |  0.5578   |  3.169    |  19.0     |  0.4125   |  26.1     |
|  7        | -1.895    |  0.6346   |  0.2918   |  0.4577   |  6.163    |  33.45    |  0.2907   |  17.51    |
|  8      

In [10]:
lgbmBO.max

{'target': -1.8888994232221157,
 'params': {'bagging_fraction': 0.5,
  'lambda_l1': 0.0,
  'lambda_l2': 1.0,
  'max_depth': 7.0,
  'min_child_samples': 17.795932537903322,
  'min_child_weight': 0.01,
  'num_leaves': 24.30000086877875}}

# Random Forest 

In [11]:
class rf_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {
            'max_depth' : 5,
            'min_samples_split' : 2,
            'min_samples_leaf': 1,
            'min_weight_fraction_leaf': 0.0,
            'min_impurity_decrease': 0.0
            
        }
        params['max_depth'] = int(param['max_depth'])
        params['min_samples_split'] = int(param['min_samples_split'])
        params['min_samples_leaf'] = int(param['min_samples_leaf'])
        params['min_weight_fraction_leaf'] = param['min_weight_fraction_leaf']
        params['min_impurity_decrease'] = param['min_impurity_decrease']

        return params
        
    def evaluate(self, max_depth, min_samples_split, min_samples_leaf,
                min_weight_fraction_leaf, min_impurity_decrease):
        params = {'max_depth':max_depth, 
                  'min_samples_split':min_samples_split, 
                  'min_samples_leaf':min_samples_leaf,
                 'min_weight_fraction_leaf':min_weight_fraction_leaf,
                 'min_impurity_decrease' : min_impurity_decrease}
        
        params = self.clean_param(params)

        rf_model = RandomForestRegressor(**params)
        rf_model.fit(self.x_train, self.y_train)
        y_pred = rf_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [12]:
rf = rf_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
rfBO = BayesianOptimization(rf.evaluate, {'max_depth': (1, 10),
                                              'min_samples_split': (10, 20),
                                              'min_samples_leaf': (5, 50),
                                            'min_weight_fraction_leaf' :(0,0.5),
                                            'min_impurity_decrease' : (0,1)
                                           }, 
                             random_state=3)

rfBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | max_depth | min_im... | min_sa... | min_sa... | min_we... |
-------------------------------------------------------------------------------------
|  1        | -2.363    |  5.957    |  0.7081   |  18.09    |  15.11    |  0.4465   |
|  2        | -2.15     |  9.067    |  0.1256   |  14.33    |  10.51    |  0.2204   |
|  3        | -2.39     |  1.269    |  0.4568   |  34.21    |  12.78    |  0.3381   |
|  4        | -2.184    |  6.318    |  0.02398  |  30.15    |  12.59    |  0.2076   |
|  5        | -2.202    |  3.552    |  0.6931   |  24.82    |  11.57    |  0.2723   |
|  6        | -2.372    |  8.023    |  0.3064   |  14.99    |  13.88    |  0.4682   |
|  7        | -2.164    |  9.784    |  0.6724   |  45.63    |  18.46    |  0.189    |
|  8        | -2.339    |  1.83     |  0.6534   |  30.1     |  13.62    |  0.1125   |
|  9        | -2.215    |  4.659    |  0.4689   |  17.12    |  12.92    |  0.2288   |
|  10       | -2.177    |  8.745    |  0.5863   |  17.

In [13]:
rfBO.max

{'target': -1.8979323609243068,
 'params': {'max_depth': 10.0,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 27.643044588529033,
  'min_samples_split': 20.0,
  'min_weight_fraction_leaf': 0.0}}

# Extra Tree 

In [14]:
class et_target :
    def __init__(self, x_train, y_train, x_test, y_test) :
        self.x_train = x_train
        self.y_train = y_train
        self.x_test = x_test
        self.y_test = y_test
        
    def clean_param(self, param) :
        params = {
            'max_depth' : 5,
            'min_samples_split' : 2,
            'min_samples_leaf': 1,
            'min_weight_fraction_leaf': 0.0,
            'min_impurity_decrease': 0.0
            
        }
        params['max_depth'] = int(param['max_depth'])
        params['min_samples_split'] = int(param['min_samples_split'])
        params['min_samples_leaf'] = int(param['min_samples_leaf'])
        params['min_weight_fraction_leaf'] = param['min_weight_fraction_leaf']
        params['min_impurity_decrease'] = param['min_impurity_decrease']

        return params
        
    def evaluate(self, max_depth, min_samples_split, min_samples_leaf,
                min_weight_fraction_leaf, min_impurity_decrease):
        params = {'max_depth':max_depth, 
                  'min_samples_split':min_samples_split, 
                  'min_samples_leaf':min_samples_leaf,
                 'min_weight_fraction_leaf':min_weight_fraction_leaf,
                 'min_impurity_decrease' : min_impurity_decrease}
        
        params = self.clean_param(params)

        et_model = ExtraTreesRegressor(**params)
        et_model.fit(self.x_train, self.y_train)
        y_pred = et_model.predict(self.x_test)
        predictions = [value for value in y_pred]
        mse = mean_squared_error(self.y_test, predictions)
        rmse = np.sqrt(mse)
        return -1*rmse

In [15]:
et = et_target(X_train_subset, y_train_subset, X_test_subset, y_test_subset)
etBO = BayesianOptimization(et.evaluate, {'max_depth': (1, 10),
                                              'min_samples_split': (10, 20),
                                              'min_samples_leaf': (5, 50),
                                            'min_weight_fraction_leaf' :(0,0.5),
                                            'min_impurity_decrease' : (0,1)
                                           }, 
                             random_state=3)

etBO.maximize(init_points=15, n_iter=10)

|   iter    |  target   | max_depth | min_im... | min_sa... | min_sa... | min_we... |
-------------------------------------------------------------------------------------
|  1        | -2.564    |  5.957    |  0.7081   |  18.09    |  15.11    |  0.4465   |
|  2        | -2.259    |  9.067    |  0.1256   |  14.33    |  10.51    |  0.2204   |
|  3        | -2.376    |  1.269    |  0.4568   |  34.21    |  12.78    |  0.3381   |
|  4        | -2.283    |  6.318    |  0.02398  |  30.15    |  12.59    |  0.2076   |
|  5        | -2.337    |  3.552    |  0.6931   |  24.82    |  11.57    |  0.2723   |
|  6        | -2.602    |  8.023    |  0.3064   |  14.99    |  13.88    |  0.4682   |
|  7        | -2.27     |  9.784    |  0.6724   |  45.63    |  18.46    |  0.189    |
|  8        | -2.406    |  1.83     |  0.6534   |  30.1     |  13.62    |  0.1125   |
|  9        | -2.359    |  4.659    |  0.4689   |  17.12    |  12.92    |  0.2288   |
|  10       | -2.291    |  8.745    |  0.5863   |  17.

In [16]:
etBO.max

{'target': -1.8652185981184386,
 'params': {'max_depth': 10.0,
  'min_impurity_decrease': 0.0,
  'min_samples_leaf': 5.0,
  'min_samples_split': 20.0,
  'min_weight_fraction_leaf': 0.0}}